In [8]:
import great_expectations as gx

In [9]:
context = gx.get_context()

In [10]:
datasource = context.get_datasource("f3_backblast")

In [11]:
asset = context.get_datasource("f3_backblast").get_asset("ao_info_peakcity")

In [ ]:
validator = context.get_validator(batch_request=asset.build_batch_request(), expectation_suite_name="ao_info")

In [ ]:
validator.expect_column_values_to_not_be_null("ao_day_of_week_int", mostly=0.40)

In [ ]:
validator.save_expectation_suite()

In [ ]:
validator.expect_table_row_count_to_be_between(5, 130, meta={"notes": {"content": "A region should only have a handful of AOs"}})

In [ ]:
validator.expect_table_columns_to_match_ordered_list(validator.columns())

In [ ]:
validator.expect_column_values_to_be_unique("ao")

In [ ]:
for column in validator.columns():
    validator.expect_column_values_to_not_be_null(column)

In [ ]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [ ]:
validator.expect_column_values_to_be_between("ao_lon", -79.5, -78)

In [ ]:
validator.expect_column_values_to_be_between("ao_lat", 35, 36)

In [ ]:
res = context.assistants.onboarding.run(batch_request=asset.build_batch_request())

In [ ]:
validator.expect_column_distinct_values_to_be_in_set("ao_type", ["1stf", "2ndf", "3rdf"])

In [ ]:
validator.expect_column_values_to_be_between("ao_day_of_week_int", 1, 6)

In [ ]:
validator.save_expectation_suite()